In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from sklearn import utils
import wandb
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy import stats

pd.set_option('display.max_columns', 500)

# local modules
from utils.config import Config
import utils.utils as utils
import utils.plots as plots
import modelsNN.modelsNN as modelsNN

In [ ]:
# Local:
PATH_DEEPSF = 'C:/Users/joseb/Downloads/deepsf/data'

In [ ]:
# %%% Setting deepsf NN characteristics.
configSF = Config()
configSF.get_config()

In [ ]:
# %%% Get the data
data_prep = utils.get_data(PATH_DEEPSF, configSF)

In [ ]:
model_name = 'deepSF2hidden.pt'
project_name = 'Training_DeepSF2hidden'

In [ ]:
# %%% Get the model
model = utils.get_model(configSF, data_prep)
optimizer = utils.build_optimizer(model, configSF.optimizer, configSF.learning_rate)

# %%% Train
history = utils.fit(configSF.epochs, 
                    model, 
                    data_prep.train_loader, 
                    data_prep.val_loader, 
                    optimizer=optimizer,
                    hyperparameters = configSF.get_config(),
                    if_wandb = configSF.if_wandb,
                    path = PATH_DEEPSF,
                    model_name = model_name,
                    project_name = project_name) 
                   
# %%% Plot results
solution = plots.plot_results(history = history, 
                     scaledTrain_df = data_prep.scaledTrain_df, 
                     train_labels = data_prep.train_labels, 
                     scaled_train_gn = data_prep.scaled_train_gn, 
                     scaledValidation_df = data_prep.scaledValidation_df, 
                     valid_labels = data_prep.valid_labels , 
                     scaled_valid_gn = data_prep.scaled_valid_gn, 
                     model = model, 
                     getBM = data_prep.getBM,
                     if_wandb = configSF.if_wandb,
                     if_ensemble = configSF.if_ensemble,
                     config = configSF)
        
if configSF.if_wandb:
    wandb.finish()

solution.solution_train_cor_total
solution.solution_train_df
solution.solution_val_cor_total
solution.solution_val_df

In [ ]:
# Deeplift
df_score_SFs = utils.do_deeplift(model, 
                 X_test = data_prep.scaledValidation_df,
                 gn_test = data_prep.scaled_valid_gn, 
                 y_test = data_prep.valid_labels, 
                 data_prep = data_prep, 
                 if_ensemble = configSF.if_ensemble)
df_score_SFs.head()